# Учебный проект 10_Прогноз заболевания сердца у пациентов

## Содержание

* [Описание проекта](#Описание)
* [Импорт библиотек Python](#Импорт)
* [Загрузка данных](#Загрузка)
* [Предобработка данных](#Предобработка)
* [Исследовательский анализ данных](#Исследование)
* [Корреляционный анализ данных](#Корреляция)
* [Построение моделей классификации данных](#Моделирование)
    * [Подготовка данных и построение пайплайна МО](#Моделирование_пайплайн)
    * [Выбор оптимальной модели](#Моделирование_обучение)
    * [Прогноз риска болезни сердца на тестовом наборе](#Моделирование_Прогноз)
* [Общий вывод](#ОбщийВывод)

На исследовании находятся данные с `информацией о пациентах клиники и наличием у них заболеваний сердца`, которые были взяты из открытого источника Kaggle.


---

`Задача`

1. Разработать и выбрать лучшую модель машинного обучения для **прогноза у пациента высокого или низкого риска поражения сердца**;
2. Разработать **приложение для автоматизированного прогнозирования риска поражения сердца на FastAPI**.

---

`Путь решения`

1. Собрать данные по сотрудникам в следующем ключе:
    * Данные о пациентах клиники - конкретно об их физиологических показателях и образе жизни:
        * Данные для обучения моделей классификации;
        * Данные для тестирования качества моделей.
2. Исследовать датасеты на предмет лучшего понимания сути данных;
3. Провести предобработку значений в наборах данных;
4. Провести исследовательский анализ данных для выявления закономерностей, применимых к последующей настройке моделей МО;
5. Построить и выбрать лучшую модель, которая `выполнит прогноз высокого или низкого риска поражения сердца` (задача может решаться моделями классификации);
6. Оценить качество моделей на метриках. Оценить важность признаков для получения результатов прогноза;
7. Сформировать вывод о подготовленных решениях. Предложить заказчику лучшую модели для решения обозначенной задачи;
8. Подготовить скрипты и библиотеки для автоматизированной обработки данных и построения прогноза в приложении.

---

`Располагаемые данные`

**Данные обучающей и тестовой выборок (медицинские показатели и другая информация о пациентах) - train_data и test_features**

Нормализованные количественные признаки: 
* Age - возраст пациента;
* Cholesterol — содержание холестерина в крови пациента;
* Heart rate — частота сердечных сокращений пациента;
* Exercise Hours Per Week - количество часов физической активности;
* Sedentary Hours Per Day - количество часов в сидячем положении;
* Income - доход пациента;
* BMI - индекс массы тела пациента;
* Sleep Hours Per Day - количество часов сна в день;
* Triglycerides - содержание жиров в теле пациента;
* Blood sugar - содержание сахара в крови;
* CK-MB - содержание креатинкиназы в крови;
* Troponin - содержание тропонина;
* Systolic blood pressure - систолическое кровяное давление;
* Diastolic blood pressure - диастолическое кровяное давление;

Бинарные признаки:
* Diabetes — наличие диабета у пациента;
* Family History — наличие семейной истории сердечных заболеваний у пациента;
* Smoking — наличие привычки курения у пациента;
* Obesity — наличие ожирения у пациента;
* Alcohol Consumption — потребляет ли пациент алкоголь;
* Previous Heart Problems - наличие у пациента прошлых проблем с сердцем;
* Medication use - принимает ли пациент медицинские препараты;
* Gender - пол пациента;
* Heart Attack Risk (Binary) - наличие риска сердечного заболевания (целевой признак);

Категориальные признаки:
* Diet — тип питания пациента;
* Stress Level - уровень стресса пациента;
* Physical Activity Days Per Week - количество дней физической активности в неделю;

Прочие показатели:
* id - уникальный идентификатор пациента;

## Импорт библиотек Python <a class = 'anchor' id = 'Импорт'></a>

1. Импорт библиотек Python:
    * для манипулирования данными;
    * для визуализации данных;
    * для решения задач машинного обучения:
        * модели классификации;
        * метрики оценки эффективности моделей;
        * механизмы отбора данных и подбора параметров моделей;
        * механизмы подготовки данных;
        * механизмы построения пайплайнов;
        * механизм заполнения пустых значений;
        * механизм анализха влияния признаков.
2. Инициализация переменных-констант для последующего использования на этапе построения моделей МО;
3. Формирование вывода по итогам данного этапа.

In [ ]:
# импорт библиотек python

# для манипулирования данными
import pandas as pd
# установка параметров для отображения табличных данных
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_info_columns', 30)
pd.set_option('display.max_rows', 5)
pd.set_option('display.max_colwidth', 100)
# установка параметров для отображения числовых значений
pd.set_option('display.float_format', '{:.2f}'.format)

# для визуализации данных
import matplotlib.pyplot as plt
import seaborn as sns
# установка размеров для последующих графиков в проекте
plt.rcParams['figure.figsize'] = (10, 5)

# для решения задач машинного обучения
# модели классификации
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from catboost import CatBoostClassifier

# метрики оценки эффективности моделей
from sklearn.metrics import (recall_score,
                             precision_score,
                             confusion_matrix,
                             roc_auc_score)

# механизмы отбора данных и подбора параметров моделей
from sklearn.model_selection import train_test_split, GridSearchCV

# механизмы построения пайплайнов
from sklearn.pipeline import Pipeline

In [ ]:
# инициализация констант для дальнейшего использования в проекте
# инициализация переменной RANDOM_STATE для фиксирования "случайности"
RANDOM_STATE = 42
TEST_SIZE = 0.25

**Вывод**

1. Импортированы библиотеки Python:
    * для манипулирования данными:
        * pandas;
        * numpy.
    * для визуализации данных:
        * matplotlib.pyplot;
        * seaborn.
    * для решения задач машинного обучения:
        * LogisticRegression - модель логистической регрессии;
        * KNeighborsClassifier - модель k-ближайших соседей;
        * SVC - машина опорных векторов;
        * DecisionTreeClassifier - модель дерева принятия решений для классификации данных;
        * AdaBoostClassifier - модель градиентного бустинга на деревьях решений;
        * CatBoostClassifier - модель градиентного бустинга на деревьях решений;
        * recall_score, precision_score, confusion_matrix, roc_auc_score - метрики оценки эффективности моделей;
        * train_test_split - механизм разделения данных;
        * GridSearchCV - механизм поиска гиперпараметров с перебором по "сетке";
        * Pipeline - механизм построения пайплайнов.
2. Инициализированы переменные **RANDOM_STATE** и **TEST_SIZE** для фиксирования "случайности" и установки размера валидационной выборки.

## Загрузка данных <a class = 'anchor' id = 'Загрузка'></a>

Данный блок характеризуется следующими последовательными действиями:

1. Загрузка данных в рабочую среду Jupyter Notebook. Инициализация переменных в соответствие с названиями загружаемых датасетов:
    * **train_data**;
    * **test_features**.
2. Вывод на экран параметров датасетов:
    * вывод общей структуры набора данных - демонстрация первых 5 строк;
    * общей информации о наборе данных;
    * визуализация распределений количественных показателей.
3. Формирование вывода по итогам данного этапа.

In [ ]:
# загрузка данных в рабочую среду

try:
    train_data = pd.read_csv('~/Desktop/YandexPraktikum_projects/datasets/heart_train.csv') # тренировочная выборка
    test_features = pd.read_csv('~/Desktop/YandexPraktikum_projects/datasets/heart_test.csv') # входные признаки тестовой выборки
except:
    train_data = pd.read_csv('/datasets/heart_train.csv')
    test_features = pd.read_csv('/datasets/heart_test.csv')

In [ ]:
# инициализация пользовательской функции для первичного изучения содержимого наборов данных
def  first_meeting (df : pd.DataFrame, df_name : str) -> None:
    print(f'Структура набора данных {df_name}')
    display(df.head())
    print('Общая информация о наборе')
    print(df.info())
    print()

In [ ]:
# инициализация пользовательской функции построения распределений количественных непрерывных показателей
def num_distribution(df : pd.DataFrame, column : str, bins : int):
    plt.subplot(1, 2, 1)
    plt.xlabel(f'Значения признака {column}')
    plt.ylabel(f'Частота значений признака')
    plt.title(f'Гистограмма значений {column}', fontsize = 10)
    sns.histplot(df, x = column, bins = bins)
    plt.subplot(1, 2, 2)
    plt.xlabel(f'Значения признака {column}')
    plt.title(f'Диаграмма размаха значений {column}', fontsize = 10)
    sns.boxplot(df, x = column)
    plt.grid(False)
    plt.show()

In [ ]:
# инициализация пользовательской функции построения диаграмм количественных дискретных показателей
def num_countplot(df : pd.DataFrame, column : str):
    sns.countplot(df, x = column)
    plt.title(f'Столбчатая диаграмма значений признака {column}', fontsize = 12)
    plt.xlabel(f'Признак {column}')
    plt.ylabel(f'Количество значений признака')
    plt.show()

In [ ]:
# вывод на экран параметров датасета 'train_data'
first_meeting(train_data, 'train_data')

# формирование множества дискретных количественных показателей
num_discrete = {'Diabetes', 'Family History', 'Smoking', 'Obesity', 'Medication Use', 'Alcohol Consumption',
            'Diet', 'Previous Heart Problems', 'Medication Use', 'Stress Level', 'Physical Activity Days Per Week', 'Heart Attack Risk (Binary)', 'Gender'}

# вывод на экран графиков дискретных количественных величин по набору 'train_data'
for col in num_discrete:
    num_countplot(train_data, col)

# вывод на экран графиков непрерывных количественных величин по набору 'train_data'
for col in set(train_data.drop(['Unnamed: 0', 'id'], axis = 1).columns).difference(num_discrete):
    num_distribution(train_data, col, 20)

In [ ]:
# проверка соответствия названий столбцов в 'train_data' и 'test_features'
print('Отсутствующие столбцы в test_features по сравнению с train_data:', set(train_data.columns.str.upper()).difference(test_features.columns.str.upper()))

In [ ]:
# удаление из переменной 'num_descrete' таргета
num_discrete.discard('Heart Attack Risk (Binary)')

# вывод на экран параметров датасета 'test_features'
first_meeting(test_features, 'test_features')

# вывод на экран графиков дискретных количественных величин по набору 'test_features'
for col in num_discrete:
    num_countplot(test_features, col)

# вывод на экран графиков непрерывных количественных величин по набору 'test_features'
for col in set(test_features.drop(['Unnamed: 0', 'id'], axis = 1).columns).difference(num_discrete):
    num_distribution(test_features, col, 20)

**Вывод**

1. Произведена загрузка данных в рабочую среду Jupyter Notebook. Инициализированы переменные в соответствие с названиями загружаемых датасетов:
    * `train_data`;
    * `test_features`.
2. Выведены на экран параметры датасетов:
    * `train_data`
        * В наборе данных **имеются пустые значения** по признакам:
            * **Diabetes**;
            * **Family History**;
            * **Smoking**;
            * **Obesity**;
            * **Alcohol Consumption**;
            * **Previous Heart Problems**;
            * **Medication Use**;
            * **Stress level**;
            * **Physical Activity Days Per Week**.
        * Признаки нуждаются в предобработке. Типы данных не соответствуют содержанию:
            * **Дискретные признаки необходимо привести к целочисленному типу**;
            * Бинарные признаки необходимо привести к единому типу - целочисленному (например, **Gender**)
        * Названия столбцов **не соответствуют формату snake_case**;
        * Наблюдаются выбросы по признакам:
            * **Heart rate**;
            * **Troponin**;
            * **Blood sugar**;
            * **CK-MB**
            
            Необходимо изучить данные случаи на этапе исследовательского анализа;
    
    * `test_features`
        * В наборе данных **имеются пустые значения** по признакам:
            * **Diabetes**;
            * **Family History**;
            * **Smoking**;
            * **Obesity**;
            * **Alcohol Consumption**;
            * **Previous Heart Problems**;
            * **Medication Use**;
            * **Stress level**;
            * **Physical Activity Days Per Week**.
        * Признаки нуждаются в предобработке. Типы данных не соответствуют содержанию:
            * **Дискретные признаки необходимо привести к целочисленному типу**;
            * Бинарные признаки необходимо привести к единому типу - целочисленному (например, **Gender**)
        * Названия столбцов **не соответствуют формату snake_case**;
        * Наблюдаются выбросы по признакам:
            * **Heart rate**;
            * **Troponin**;
            * **Blood sugar**;
            * **CK-MB**
            
            Необходимо изучить данные случаи на этапе исследовательского анализа.

## Предобработка данных <a class = 'anchor' id = 'Предобработка'></a>

Данный блок характеризуется следующими последовательными действиями:

1. Приведение заголовков наборов данных к стилю написания 'snake_case';
2. Явное приведение данных к соответствующим типам;
3. Обработка пустых значений наборов данных;
4. Проверка датасетов на дубликаты:
    * Явные дубликаты;
    * Неявные дубликаты.
5. Формирование вывода по итогам данного этапа.

In [ ]:
# инициализация пользовательской функции преобразования признаков к целочисленному типу
def float_to_int(df : pd.DataFrame, cols_list : list) -> pd.DataFrame:
    for col in cols_list:
        df[col] = df[col].astype('Int64')
    return df

In [ ]:
# замена значений в столбце 'gender' на числовые
train_data['Gender'] = train_data['Gender'].map({'Male' : 1, 'Female' : 0})
test_features['Gender'] = test_features['Gender'].map({'Male' : 1, 'Female' : 0})


# преобразование столбцов к целочисленному типу
train_data = float_to_int(train_data, num_discrete)
test_features = float_to_int(test_features, num_discrete)

display(train_data[list(num_discrete)].info())
print()
display(test_features[list(num_discrete)].info())

In [ ]:
# инициализация пользовательской функции преобразования названий столбцов к формату 'snake_case'
def snake_case_transformation(df : pd.DataFrame) -> pd.DataFrame:
    new_columns = []
    for column in df.columns:
        new_columns.append(column.lower().replace(' ', '_'))
    df.columns = new_columns
    return df

# приведение названий столбцов к формату 'snake_case'
train_data = snake_case_transformation(train_data)
test_features = snake_case_transformation(test_features)

# удаление неинформативных столбцов из наборов данных
train_data = train_data.drop('unnamed:_0', axis = 1)
test_features = test_features.drop('unnamed:_0', axis = 1)

# вывод на экран измененных названий столбцов
print('Названия столбцов train_data после преобразования:')
print(train_data.columns)
print()
print('Названия столбцов test_features после преобразования:')
print(test_features.columns)

In [ ]:
# инициализация пользовательской функции построения сводной таблицы пустых значений в наборах данных
def isna_pivot(df : pd.DataFrame):
    df_pivot = df.isna().sum().to_frame().rename(columns = {0 : 'missing_values'})
    df_pivot = df_pivot[df_pivot['missing_values'] > 0]
    df_pivot['%_share'] = (df_pivot['missing_values'] / df.shape[0] * 100).round(2)
    return df_pivot

# запись фреймов в отдельные переменные
train_data_missValues = isna_pivot(train_data)
test_features_missValues = isna_pivot(test_features)

# вывод на экран сводной таблицы пустых значений train_data
print('Сводная таблица пустых значений в train_data')
display(train_data_missValues.sort_values(by = 'missing_values', ascending = False))
print()
# вывод на экран сводной таблицы пустых значений test_features
print('Сводная таблица пустых значений в test_features')
display(test_features_missValues.sort_values(by = 'missing_values', ascending = False))

**Вывод по промежуточному этапу**

Пропущенные значения представлены в **дискретных категориальных признаках**. При этом, **большая часть признаков - бинарные**.

Пропуски в бинарных признаках возможно по той причине, что человек решил явно не указывать информацию о себе: если пациент не употребяет алкоголь (к примеру), нет смысла явно обозначать потребление "0", а можно пропустить вопрос в анкете. Подобные случаи можно заменить на "0".

Пропущенные значения в признаке **physical_activity_days_per_week** (количество дней физической активности в неделю) можно заменить на "0". Человек явно не стал указывать, что он не занимается физнагрузками в неделю.

Пропущенные значения в признаке **gender** (пол) можно заменить на "0" - пациент женского пола.

Пропущенные значения в признаке **stress_level** не получится заменить на "0". Предлагается заменить пропущенные значения на медианное по датасету.

In [ ]:
# инициализация пользовательской функции по замене пропущенных значений в признаках датасетов
def fill_miss_values(df : pd.DataFrame, cols_list : list, fill_value) -> pd.DataFrame:
    for col in cols_list:
        df[col] = df[col].fillna(fill_value)
    return  df

In [ ]:
# список столбцов с пропущенными значениями с исключением признака 'stress_level'
missValue_cols = test_features_missValues.index.to_list()
missValue_cols.remove('stress_level')

# замена пропущенных значений в наборах данных
train_data = fill_miss_values(train_data, missValue_cols, 0)
test_features = fill_miss_values(test_features, missValue_cols, 0)

# замена пропущенных значений в признаке 'stress_level'
train_data['stress_level'] = train_data['stress_level'].fillna(train_data['stress_level'].median())
test_features['stress_level'] = test_features['stress_level'].fillna(test_features['stress_level'].median())

In [ ]:
# проверка датасетов на явные дубликаты
print('Количество явных дубликатов в train_data:', train_data.duplicated().sum())
print('Количество явных дубликатов в test_features:', test_features.duplicated().sum())
print()
# дополнительная проверка на дубликаты по столбцу 'id'
print('Количество явных дубликатов в train_data при проверке признака id:', train_data['id'].duplicated().sum())
print('Количество явных дубликатов в test_features при проверке признака id:', test_features['id'].duplicated().sum())

**Вывод по промежуточному этапу**

**В датасетах отсутствуют явные дубликаты** - проверка строк целиком и признака **id** подтвердила это, все пациенты уникальны.

**Неявных дубликатов в датасетах нет** - характер и типы данных не предполагает их наличия.

**Вывод**

1. Заголовки наборов данных **приведены к стилю написания 'snake_case'**;
2. Проведено преобразование типов данных. Дискретные количественнные признаки преобразованы к целочисленному типу;
3. Проведена обработка пропущенных значений в столбцах наборов данных. Большая часть пропусков была заменена на "0" по причине того, что пациент решил явно не указывать о себе информацию в анкете. Пропущенные значения по признаку **stress_rate** были заменены на медианное значение по датасету;
4. Выполнена проверка наборов данных на дубликаты:
    * **Явные дубликаты** - явные дубликаты **не обнаружены**; 
    * **Неявные дубликаты** - неявные дубликаты **не обнаружены**. 
5. Данные прошли этап предобработки и готовы к исследовательскому анализу.

## Исследовательский анализ данных <a class = 'anchor' id = 'Исследование'></a>

Данный блок характеризуется следующими последовательными действиями:

1. Исследование соотношения количества значений категориальных показателей в наборе `train_data`;
2. Исследование распределения количественных показателей набора `train_data`. Построение **гистограмм распределения значений**;
3. Формирование вывода по итогам этапа.

### Исследование категориальных признаков <a class = 'anchor' id = 'Исследование_категория'></a>

In [ ]:
# инициализация пользовательской функции по формированию вывода информации

def display_info(df: pd.DataFrame, column_name: str, title: str, xlabel: str, kind_of_plot : str):
# построение визуализации по выбранной метрике
    plt.title(title, fontsize = 12)
    if kind_of_plot == 'pie':
        (df[column_name]
         .value_counts()
         .sort_values(ascending=True)
         .plot(kind = kind_of_plot, figsize = (8, 5), autopct='%1.0f%%'))
    else:
        (df[column_name]
         .value_counts()
         .sort_values(ascending=True)
         .plot(kind = kind_of_plot, figsize = (8, 5)))
    ax = plt.gca()
    ax.axes.yaxis.set_visible(False)
    plt.xlabel(xlabel)
    plt.show()

# построение сводной таблицы по выбранной метрике
    pivot_data = (df[column_name]
                  .value_counts()
                  .sort_values(ascending=False)
                  .to_frame())
    pivot_data['share_of_patients'] = round(pivot_data['count'] / pivot_data['count'].sum() * 100, 2)
    pivot_data.columns = ['count_of_patients', 'share_of_patients']
    display(pivot_data)

In [ ]:
# инициализация пользовательской функции по построению гистограмм по передаваемым метрикам
def histogram_plotting(data: pd.DataFrame, feature : str, bins: int, x_size: int, y_size: int, feature_xlabel : str):
    # вычисление статистических метрик для дальнейшей визуализации
    q1 = data[feature].quantile(0.25)
    q3 = data[feature].quantile(0.75)
    upper_bound = q3 + 1.5 * (q3 - q1)
    lower_bound = q1 - 1.5 * (q3 - q1)

    # построение визуализации
    plt.figure(figsize = (x_size, y_size))
    plt.hist(data[feature], color = 'blue', edgecolor = 'white', bins = bins)
    plt.axvline(upper_bound, c = 'red', ls = '-', label = 'верхняя граница допустимых значений')
    plt.axvline(q3, c = 'red', ls = '--', label = '3 квартиль значений')
    plt.axvline(q1, c = 'black', ls = '--', label = '1 квартиль значений')
    plt.axvline(lower_bound, c = 'black', ls = '-', label = 'нижняя граница допустимых значений')
    plt.title(f'Гистограмма распределения значений по метрике: {feature_xlabel}', fontsize = 10)
    plt.xlabel(feature_xlabel)
    plt.ylabel('Количество значений по метрике')
    plt.legend(bbox_to_anchor = (1, 0.6))
    plt.show()

    # вывод статистических метрик на экран
    print('Верхняя допустимая граница значений:', upper_bound)
    print('Нижняя допустимая граница значений:', lower_bound)
    print('Медианное значение:', data[feature].median())
    print('Среднее значение:', round(data[feature].mean(), 2))

    # расчет доли аномальных значений по метрике
    print('Доля значений, выходящих за верхнюю границу: {:.2%}'.format(data[data[feature] > upper_bound].shape[0] / data[feature].shape[0]))
    print('Доля значений, выходящих за нижнюю границу: {:.2%}'.format(data[data[feature] < lower_bound].shape[0] / data[feature].shape[0]))

In [ ]:
# построение пай-чарта по соотношению пациентов в контексте наличия диабета
display_info(train_data, 'diabetes', 'Соотношение пациентов по наличию диабета', 'Наличие диабета', 'pie')

In [ ]:
# построение пай-чарта по соотношению пациентов в контексте наличия семейной истории
display_info(train_data, 'family_history', 'Соотношение пациентов по наличию семейной истории', 'Наличие семейной истории', 'pie')

In [ ]:
# построение пай-чарта по соотношению пациентов в контексте наличия привычки курения
display_info(train_data, 'smoking', 'Соотношение пациентов по наличию привычки курения', 'Наличие привычки курения', 'pie')

In [ ]:
# построение пай-чарта по соотношению пациентов в контексте наличия ожирения
display_info(train_data, 'obesity', 'Соотношение пациентов по наличию ожирения', 'Наличие ожирения', 'pie')

In [ ]:
# построение пай-чарта по соотношению пациентов в контексте потребления алкоголя
display_info(train_data, 'alcohol_consumption', 'Соотношение пациентов по потреблению алкоголя', 'Потребление алкоголя', 'pie')

In [ ]:
# построение пай-чарта по соотношению пациентов в контексте наличия предыдущих заболеваний сердца
display_info(train_data, 'previous_heart_problems', 'Соотношение пациентов по наличию предыдущих заболеваний', 'Наличие предыдущих заболеваний', 'pie')

In [ ]:
# построение пай-чарта по соотношению пациентов в контексте применения препаратов
display_info(train_data, 'medication_use', 'Соотношение пациентов по применению препаратов', 'Применение препаратов', 'pie')

In [ ]:
# построение пай-чарта по соотношению пациентов в контексте пола
display_info(train_data, 'gender', 'Соотношение пациентов по полу', 'Пол', 'pie')


In [ ]:
# построение пай-чарта по соотношению пациентов в контексте риска развития заболеваний сердца
display_info(train_data, 'heart_attack_risk_(binary)', 'Соотношение пациентов по риску развития заболеваний сердца', 'Риск развития заболеваний сердца', 'pie')

In [ ]:
# построение пай-чарта по соотношению пациентов в контексте типов питания пациентов
display_info(train_data, 'diet', 'Соотношение пациентов по типу питания', 'Тип питания', 'pie')

In [ ]:
# построение пай-чарта по соотношению пациентов в контексте уровня стресса
display_info(train_data, 'stress_level', 'Соотношение пациентов по уровню стресса', 'Уровень стресса', 'pie')

In [ ]:
# построение пай-чарта по соотношению пациентов в контексте уровня физической активности
display_info(train_data, 'physical_activity_days_per_week', 'Соотношение пациентов по уровню физической активности', 'Уровень физической активности', 'pie')

**Вывод по промежуточному этапу**

В целом, значения категориальных признаков в наборе `train_data` соотнесены в равных пропорциях.

Явный дисбаланс значений наблюдается в следующих признаках:
* **gender** - соотношение пациентов по полу: **68% мужчин и 32% женщин**;
* **diabetes** - соотношение пациентов по наличию диабета: **63% с диабетом и 37% без**;
* **smoking** - соотношение пациентов по привычке курения: **88% курящих и 12% не курящих**;
* **diet** - соотношение пациентов по типу диеты: **3 типа диеты придерживаются только 3% пациентов, остальные 97% - распределены по другим типам диет**;
* **heart_attack_risk_(binary)** - соотношение пациентов по риску развития заболеваний сердца: **65% пациентов подвержены риску заболевания и 35% - нет**.

Таким образом, подобное соотношение значений в признаках и целевой переменной накладывает определенные ограничения на выбор моделей классификации данных и метрик их оценки:
* ROC-AUC;
* F1-мера;
* Confusion Matrix;

### Исследование количественных признаков <a class = 'anchor' id = 'Исследование_количество'></a>

In [ ]:
# построение  гистограммы распределения значений по признаку 'age'
histogram_plotting(train_data, 'age', 20, 10, 5, 'Возраст пациента')

In [ ]:
# построение  гистограммы распределения значений по признаку 'cholesterol'
histogram_plotting(train_data, 'cholesterol', 20, 10, 5, 'Содержание холестерина в крови')

In [ ]:
# построение  гистограммы распределения значений по признаку 'heart_rate'
histogram_plotting(train_data, 'heart_rate', 20, 10, 5, 'Пульс пациента')

In [ ]:
# построение  гистограммы распределения значений по признаку 'exercise_hours_per_week'
histogram_plotting(train_data, 'exercise_hours_per_week', 20, 10, 5, 'Количество часов тренировок в неделю')

In [ ]:
# построение  гистограммы распределения значений по признаку 'sedentary_hours_per_day'
histogram_plotting(train_data, 'sedentary_hours_per_day', 20, 10, 5, 'Количество часов в сидячем положении')

In [ ]:
# построение  гистограммы распределения значений по признаку 'income'
histogram_plotting(train_data, 'income', 20, 10, 5, 'Доход пациента')

In [ ]:
# построение  гистограммы распределения значений по признаку 'bmi'
histogram_plotting(train_data, 'bmi', 20, 10, 5, 'Индекс массы тела')

In [ ]:
# построение  гистограммы распределения значений по признаку 'sleep_hours_per_day'
histogram_plotting(train_data, 'sleep_hours_per_day', 20, 10, 5, 'Количество часов сна в день')

In [ ]:
# построение  гистограммы распределения значений по признаку 'triglycerides'
histogram_plotting(train_data, 'triglycerides', 20, 10, 5, 'Содержание жиров в теле пациента')

In [ ]:
# построение  гистограммы распределения значений по признаку 'blood_sugar'
histogram_plotting(train_data, 'blood_sugar', 20, 10, 5, 'Уровень сахара в крови')

In [ ]:
# построение  гистограммы распределения значений по признаку 'ck-mb'
histogram_plotting(train_data, 'ck-mb', 20, 10, 5, 'Содержание креатинкиназы в крови')

In [ ]:
# построение  гистограммы распределения значений по признаку 'troponin'
histogram_plotting(train_data, 'troponin', 20, 10, 5, 'Уровень тропонина')

In [ ]:
# построение  гистограммы распределения значений по признаку 'systolic_blood_pressure'
histogram_plotting(train_data, 'systolic_blood_pressure', 20, 10, 5, 'Систолическое давление')

In [ ]:
# построение  гистограммы распределения значений по признаку 'diastolic_blood_pressure'
histogram_plotting(train_data, 'diastolic_blood_pressure', 20, 10, 5, 'Диастолическое давление')

**Вывод по промежуточному этапу**

В целом, нормализованные количественные показатели находятся в границах допустимых значений.

Аномально большие или минимальные значения наблюдаются в следующих признаках:
* **heart_rate** - доля значений, **выходящих за верхнюю границу: 0.02%**;
* **blood_sugar** - доля значений, **выходящих за нижнюю границу: 16.34%**. Доля значений, **выходящих за верхнюю границу: 8.23%**;
* **ck-mb** - доля значений, **выходящих за нижнюю границу: 21.21%**. Доля значений, **выходящих за верхнюю границу: 3.36%**;
* **troponin** - доля значений, **выходящих за нижнюю границу: 20.29%**. Доля значений, **выходящих за верхнюю границу: 4.28%**;

Необходимо исключить аномально большие значения признаков из набора данных: в признаках **ck-mb** и **heart_rate**, где наблюдаются особо сильные отклонения от общей массы. Значения признаков, выходящих за нижнюю границу, можно оставить. Скорее всего, они являются результатом индивидуальных особенностей организма пациента.

**Дополнительных преобразований количественных значений не требуется**

In [ ]:
# фильтрация набора данных от аномально больших значений
train_data = train_data[train_data['heart_rate'] <= 0.2]
train_data = train_data[train_data['ck-mb'] < 1.0]


**Вывод**

1. Исследование соотношения количества значений качественных показателей
    
    В целом, значения категориальных признаков в наборе `train_data` соотнесены в равных пропорциях.

    Явный дисбаланс значений наблюдается в следующих признаках:
    * **gender** - соотношение пациентов по полу: **68% мужчин и 32% женщин**;
    * **diabetes** - соотношение пациентов по наличию диабета: **63% с диабетом и 37% без**;
    * **smoking** - соотношение пациентов по привычке курения: **88% курящих и 12% не курящих**;
    * **diet** - соотношение пациентов по типу диеты: **3 типа диеты придерживаются только 3% пациентов, остальные 97% - распределены по другим типам диет**;
    * **heart_attack_risk_(binary)** - соотношение пациентов по риску развития заболеваний сердца: **65% пациентов подвержены риску заболевания и 35% - нет**.

    Таким образом, подобное соотношение значений в признаках и целевой переменной накладывает определенные ограничения на выбор моделей классификации данных и метрик их оценки:
    * ROC-AUC;
    * F1-мера;
    * Confusion Matrix;
2. Исследование распределения количественных показателей наборов. Построение **гистограмм распределения значений**:
    В целом, нормализованные количественные показатели находятся в границах допустимых значений.

    Аномально большие или минимальные значения наблюдаются в следующих признаках:
    * **heart_rate** - доля значений, **выходящих за верхнюю границу: 0.02%**;
    * **blood_sugar** - доля значений, **выходящих за нижнюю границу: 16.34%**. Доля значений, **выходящих за верхнюю границу: 8.23%**;
    * **ck-mb** - доля значений, **выходящих за нижнюю границу: 21.21%**. Доля значений, **выходящих за верхнюю границу: 3.36%**;
    * **troponin** - доля значений, **выходящих за нижнюю границу: 20.29%**. Доля значений, **выходящих за верхнюю границу: 4.28%**;

    **Дополнительных преобразований количественных значений не требуется**
3. Произведена фильтрация набора данных `train_data` - исключены аномально большие значения.

## Корреляционный анализ данных <a class = 'anchor' id = 'Корреляция'></a>

Данный блок характеризуется следующими последовательными действиями:

1. Построение матрицы корреляции - поиск признаков высокой взаимосвязи показателей объектов;
2. Проведение отбора признаков для последующего построения моделей машинного обучения;
3. Формирование вывода по итогам данного этапа.

In [ ]:
# построение матрицы корреляции и поиск сильных взаимосвязей
plt.figure(figsize=(25, 8))
sns.heatmap(train_data.drop('id', axis = 1).corr(method = 'spearman').round(2), annot = True)
plt.title('Матрица корреляции Спирмана между количественными показателями набора данных', fontsize = 12)
plt.show()

In [ ]:
# исключение из наборов данных показателей 'troponin' и 'gender'
train_data = train_data.drop(['troponin', 'gender'], axis = 1)
test_features = test_features.drop(['troponin', 'gender', 'id'], axis = 1)

**Вывод:**

1. Построена матрица корреляции Спирмана между признаками набора данных и целевой переменной;
2. Обнаружена умеренная взаимосвязь между признаками:
    * **gender** и **smoking** - 0.54;
    * **troponin** и **ck-mb** - 0.45

Из набора исключены признаки **troponin** и **gender**.

3. Сильно коррелирующих признаков между собой и целевой переменной **не обнаружено**. Набор данных готов к последующему использованию в алгоритмах моделей машинного обучения.

## Построение моделей классификации данных <a class = 'anchor' id = 'Моделирование'></a>

### Подготовка данных и построение пайплайна МО <a class = 'anchor' id = 'Моделирование_пайплайн'></a>

Данный блок характеризуется следующими последовательными действиями:

1. Разделение исходного набора - `train_data` - на область признаков и вектор целевой переменной - инициализация переменных **X** и **y** соответственно;
2. Формирование обучающей и тестовой выборок в соотношении **75:25** - инициализация переменных **X_train**, **X_test**, **y_train**, **y_valid**;
3. Формирование пайплайна обработки данных и обучения моделей классификации данных;
4. Инициализация переменной **param_distributions** для хранения моделей и их гиперпараметров для последующего выбора лучшей;
5. Формирование вывода по итогам данного этапа.

In [ ]:
# формирование области признаков и вектора целевой переменной
X = train_data.drop(['id', 'heart_attack_risk_(binary)'], axis = 1)
y = train_data['heart_attack_risk_(binary)']

In [ ]:
# формирование обучающей и тестовой выборок
X_train, X_valid, y_train, y_valid = train_test_split(
    X,
    y,
    random_state = RANDOM_STATE,
    test_size = TEST_SIZE,
    stratify = y
)

In [ ]:
# проверка размерности получившихся наборов
if X_train.shape[1] == X_valid.shape[1]:
    print(f'Количество столбцов {X_train.shape[1]}. Потери признаков при разделении исходного набора не произошло')
else:
    print(f'Произведено неверное разделение данных. Зафиксирована утечка признаков')
    
if round(X_valid.shape[0] / X.shape[0], 2) == 0.25:
    print(f'Соотношение размера тестовой выборки к исходному набору - 0.25. Соблюдено верное соотношение разделения данных')
else:
    print(f'Произведено неверное разделение данных. Зафиксирована утечка строк')

In [ ]:
# формирование пайплайна подготовки данных и обучения модели - используется модель DecisionTreeClassifier
final_pipeline = Pipeline(
    [
        ('models', DecisionTreeClassifier(random_state = RANDOM_STATE))
    ]
)

In [ ]:
# инициализация переменной 'param_grid' для поиска оптимальной модели
param_grid = {
    'models' : [DecisionTreeClassifier(random_state = RANDOM_STATE),
                KNeighborsClassifier(),
                SVC(random_state = RANDOM_STATE),
                LogisticRegression(random_state = RANDOM_STATE),
                AdaBoostClassifier(random_state = RANDOM_STATE),
                CatBoostClassifier(random_state = RANDOM_STATE)]
}

In [ ]:
# инициализация переменной 'param_distributions' для хранения распределения значений гиперпараметров
param_distributions = [
    {
        'models' : [KNeighborsClassifier()],
        'models__n_neighbors' : range(1, 20)
    },
    {
        'models' : [DecisionTreeClassifier(random_state = RANDOM_STATE)],
        'models__max_depth' : range(2, 11),
        'models__min_samples_split' : range(2, 6),
        'models__min_samples_leaf' : range(2, 6)
    },
    {
        'models' : [SVC(random_state = RANDOM_STATE, probability = True)]
    },
    {
        'models' : [LogisticRegression(random_state = RANDOM_STATE)]
    },
    {
        'models' : [AdaBoostClassifier(random_state = RANDOM_STATE)],
        'models__n_estimators' : range(40, 50)
    },
    {
        'models' : [CatBoostClassifier(random_state = RANDOM_STATE)],
        'models__depth' : range(5, 10)
    }
    
]

Данный блок характеризуется следующими последовательными действиями:

1. Проведено разделение исходного набора - `train_data` - на область признаков и вектор целевой переменной - инициализация переменных **X** и **y** соответственно;
2. Сформированы обучающая и валидационная выборки в соотношении **75:25** - инициализация переменных **X_train**, **X_valid**, **y_train**, **y_valid**;
3. Проведено формирование пайплайна обработки данных и обучения моделей классификации данных. Инициализирована переменная **final_pipeline**;
4. Инициализирована переменная **param_distributions** для хранения моделей и их гиперпараметров для последующего выбора лучшей.

### Выбор оптимальной модели <a class = 'anchor' id = 'Моделирование_обучение'></a>

Данный блок характеризуется следующими последовательными действиями:

1. Выбор оптимальной модели из набора с перебором параметров и поиском по сетке - **GridSearchCV**. Вывод на экран наименования лучшей модели и значения метрики качества;
2. Построение матрицы ошибок для оценки эффективности модели;
3. Формирование вывода по итогам данного этапа.

In [ ]:
grid_withParams = GridSearchCV(
    final_pipeline,
    param_grid = param_distributions,
    cv = 5,
    scoring = 'recall',
    n_jobs = -1
)

# обучение модели на тренировочном наборе
grid_withParams.fit(X_train, y_train)

# вывод лучшей модели на экран
print('Лучшая модель классификации и ее параметры')
print(grid_withParams.best_params_)
print()
print('Метрика RECALL для лучшей модели')
print(round(grid_withParams.best_score_, 2))

In [ ]:
# проверка качества модели на валидационной выборке
y_pred = grid_withParams.predict_proba(X_valid)
print('Метрика RECALL для лучшей модели на ВАЛИДАЦИОННОЙ выборке:', round(recall_score(y_valid, y_pred[:, 1]), 2))
print('Метрика ROC_AUC для лучшей модели на ВАЛИДАЦИОННОЙ выборке:', round(roc_auc_score(y_valid, y_pred[:, 1]), 2))

In [ ]:
# построение матрицы ошибок
cm = confusion_matrix(y_valid, grid_withParams.predict(X_valid))

print('Метрика Precision для результатов модели K-ближайших соседей:', round(precision_score(y_valid, grid_withParams.predict(X_valid)), 2))
print('Метрика Recall для результатов модели K-ближайших соседей:', round(recall_score(y_valid, grid_withParams.predict(X_valid)), 2))


sns.heatmap(cm, annot = True, fmt= 'd', cmap = 'Blues_r')
plt.ylabel('Истинные значения')
plt.xlabel('Прогнозные значения')
plt.title('Матрица ошибок результатов модели K-ближайших соседей')
plt.show()

**Вывод**

1. Произведен выбор оптимальной модели из набора С перебором параметров и поиском по сетке - **GridSearchCV**. Лучшая модель и ее параметры:
    * **KNeighborsClassifier**;
    * **n_neighbors = 1**.
2. Построена матрица ошибок для оценки эффективности модели:
    * Значения TP - 325;
    * Значения TN - 979;
    * Значения FP - 434;
    * Значения FN - 426;
    * Всего значений в выборке - 2 164.
3. Произведена оценка качества работы модели KNeighborsClassifier на тестовой выборке:
    * Precision - 0.43;
    * Recall - 0.43;
    * ROC-AUC - 0.56.


При выборе в качестве метрики оптимизации и поиска лучшей модели классификации **RECALL** лучшей моделью **определяется поиск K-ближайших соседей**.

Модель примерно в одинаковой степени раздает метки наличия и отсутствия у пациентов сердечных заболеваний на основании схожести их медицинских показателей.

При этом, перед обучением моделей из набора данных были исключены признаки:
* **troponin**, имевший заметную корреляцию с признаком **ck-mb**;
* **gender**, имевший заметную корреляцию с признаком **smoking**.

Исключение показателя **gender** было призвано исключить привязку модели к полу пациента, так как большинство представленных пациентов в наборе - это мужчины; и большинство курящих пациентов - это так же, мужчины.

----

При этом, альтернативным вариантом выбора лучшей модели была оптимизация **метрики ROC_AUC**. В данном случае лучшей моделью выбиралась **CatBoost**.

Данный вариант имел отличные показатели по классификации ИСТИННО ЛОЖНЫХ результатов, однако данная модель давала плохие результаты по выявлению болезни сердца у пациента - выдавала ложно отрицательные результаты, когда на самом деле пациент имел проблемы с сердцем.

Понимая, что от подобных результатов модели классификации зависят жизни пациентов, было принято отказаться от модели **CatBoost**, несмотря на удовлетворительные статистические показатели.

### Прогноз риска болезни сердца на тестовом наборе <a class = 'anchor' id = 'Моделирование_Прогноз'></a>

Данный блок характеризуется следующими последовательными действиями:

1. Построение прогноза риска развития сердечных заболеваний на наборе `test_features` с помощью лучшей модели, выбранной на GridSearch. Инициализация переменной **predictions**;
2. Выгрузка прогнозных значений в директорию проекта в формате .csv

In [ ]:
# построение прогноза на тестовой выборке
predictions = grid_withParams.predict(test_features)

# сохранение получившегося прогноза в файл .csv
np.savetxt('heart_attack_risk.csv', predictions , delimiter = ',')

**Вывод**

1. Построен прогноз риска развития сердечных заболеваний на наборе `test_features` с помощью лучшей модели, выбранной на GridSearch;
2. Произведена выгрузка прогнозных значений в директорию проекта в формате .csv

## Общий вывод <a class = 'anchor' id = 'Вывод'></a>

1. Импортированы библиотеки Python:
    * для манипулирования данными:
        * pandas;
        * numpy.
    * для визуализации данных:
        * matplotlib.pyplot;
        * seaborn.
    * для решения задач машинного обучения:
        * LogisticRegression - модель логистической регрессии;
        * KNeighborsClassifier - модель k-ближайших соседей;
        * SVC - машина опорных векторов;
        * DecisionTreeClassifier - модель дерева принятия решений для классификации данных;
        * AdaBoostClassifier - модель градиентного бустинга на деревьях решений;
        * CatBoostClassifier - модель градиентного бустинга на деревьях решений;
        * recall_score, precision_score, confusion_matrix, roc_auc_score - метрики оценки эффективности моделей;
        * train_test_split - механизм разделения данных;
        * GridSearchCV - механизм поиска гиперпараметров с перебором по "сетке";
        * Pipeline - механизм построения пайплайнов.
2. Инициализированы переменные **RANDOM_STATE** и **TEST_SIZE** для фиксирования "случайности" и установки размера валидационной выборки;
3. Произведена загрузка данных в рабочую среду Jupyter Notebook. Инициализированы переменные в соответствие с названиями загружаемых датасетов:
    * `train_data`;
    * `test_features`.
4. Выведены на экран параметры датасетов:
    * `train_data`
        * В наборе данных **имеются пустые значения** по признакам:
            * **Diabetes**;
            * **Family History**;
            * **Smoking**;
            * **Obesity**;
            * **Alcohol Consumption**;
            * **Previous Heart Problems**;
            * **Medication Use**;
            * **Stress level**;
            * **Physical Activity Days Per Week**.
        * Признаки нуждаются в предобработке. Типы данных не соответствуют содержанию:
            * **Дискретные признаки необходимо привести к целочисленному типу**;
            * Бинарные признаки необходимо привести к единому типу - целочисленному (например, **Gender**)
        * Названия столбцов **не соответствуют формату snake_case**;
        * Наблюдаются выбросы по признакам:
            * **Heart rate**;
            * **Troponin**;
            * **Blood sugar**;
            * **CK-MB**.
    * `test_features`
        * В наборе данных **имеются пустые значения** по признакам:
            * **Diabetes**;
            * **Family History**;
            * **Smoking**;
            * **Obesity**;
            * **Alcohol Consumption**;
            * **Previous Heart Problems**;
            * **Medication Use**;
            * **Stress level**;
            * **Physical Activity Days Per Week**.
        * Признаки нуждаются в предобработке. Типы данных не соответствуют содержанию:
            * **Дискретные признаки необходимо привести к целочисленному типу**;
            * Бинарные признаки необходимо привести к единому типу - целочисленному (например, **Gender**)
        * Названия столбцов **не соответствуют формату snake_case**;
        * Наблюдаются выбросы по признакам:
            * **Heart rate**;
            * **Troponin**;
            * **Blood sugar**;
            * **CK-MB**.
5. Заголовки наборов данных **приведены к стилю написания 'snake_case'**;
6. Проведено преобразование типов данных. Дискретные количественнные признаки преобразованы к целочисленному типу;
7. Проведена обработка пропущенных значений в столбцах наборов данных. Большая часть пропусков была заменена на "0" по причине того, что пациент решил явно не указывать о себе информацию в анкете. Пропущенные значения по признаку **stress_rate** были заменены на медианное значение по датасету;
8. Выполнена проверка наборов данных на дубликаты:
    * **Явные дубликаты** - явные дубликаты **не обнаружены**; 
    * **Неявные дубликаты** - неявные дубликаты **не обнаружены**.
9. Исследование соотношения количества значений качественных показателей
    
    В целом, значения категориальных признаков в наборе `train_data` соотнесены в равных пропорциях.

    Явный дисбаланс значений наблюдается в следующих признаках:
    * **gender** - соотношение пациентов по полу: **68% мужчин и 32% женщин**;
    * **diabetes** - соотношение пациентов по наличию диабета: **63% с диабетом и 37% без**;
    * **smoking** - соотношение пациентов по привычке курения: **88% курящих и 12% не курящих**;
    * **diet** - соотношение пациентов по типу диеты: **3 типа диеты придерживаются только 3% пациентов, остальные 97% - распределены по другим типам диет**;
    * **heart_attack_risk_(binary)** - соотношение пациентов по риску развития заболеваний сердца: **65% пациентов подвержены риску заболевания и 35% - нет**.

    Таким образом, подобное соотношение значений в признаках и целевой переменной накладывает определенные ограничения на выбор моделей классификации данных и метрик их оценки:
    * ROC-AUC;
    * F1-мера;
    * Confusion Matrix;
10. Исследование распределения количественных показателей наборов. Построение **гистограмм распределения значений**:
    В целом, нормализованные количественные показатели находятся в границах допустимых значений.

    Аномально большие или минимальные значения наблюдаются в следующих признаках:
    * **heart_rate** - доля значений, **выходящих за верхнюю границу: 0.02%**;
    * **blood_sugar** - доля значений, **выходящих за нижнюю границу: 16.34%**. Доля значений, **выходящих за верхнюю границу: 8.23%**;
    * **ck-mb** - доля значений, **выходящих за нижнюю границу: 21.21%**. Доля значений, **выходящих за верхнюю границу: 3.36%**;
    * **troponin** - доля значений, **выходящих за нижнюю границу: 20.29%**. Доля значений, **выходящих за верхнюю границу: 4.28%**;

    **Дополнительных преобразований количественных значений не требуется**
11. Произведена фильтрация набора данных `train_data` - исключены аномально большие значения;
12. Построена матрица корреляции Спирмана между признаками набора данных и целевой переменной;
13. Обнаружена умеренная взаимосвязь между признаками:
    * **gender** и **smoking** - 0.54;
    * **troponin** и **ck-mb** - 0.45

Из набора исключены признаки **troponin** и **gender**.

14. Сильно коррелирующих признаков между собой и целевой переменной **не обнаружено**. Набор данных подготовлен к последующему использованию в алгоритмах моделей машинного обучения.
15. Проведено разделение исходного набора - `train_data` - на область признаков и вектор целевой переменной - инициализация переменных **X** и **y** соответственно;
16. Сформированы обучающая и валидационная выборки в соотношении **75:25** - инициализация переменных **X_train**, **X_valid**, **y_train**, **y_valid**;
17. Проведено формирование пайплайна обработки данных и обучения моделей классификации данных. Инициализирована переменная **final_pipeline**;
18. Инициализирована переменная **param_distributions** для хранения моделей и их гиперпараметров для последующего выбора лучшей;
19. Произведен выбор оптимальной модели из набора С перебором параметров и поиском по сетке - **GridSearchCV**. Лучшая модель и ее параметры:
    * **KNeighborsClassifier**;
    * **n_neighbors = 1**.
20. Построена матрица ошибок для оценки эффективности модели:
    * Значения TP - 325;
    * Значения TN - 979;
    * Значения FP - 434;
    * Значения FN - 426;
    * Всего значений в выборке - 2 164.
21. Произведена оценка качества работы модели KNeighborsClassifier на тестовой выборке:
    * Precision - 0.43;
    * Recall - 0.43;
    * ROC-AUC - 0.56.
22. Произведена выгрузка результатов прогнозирования риска заболеваний сердца на наборе `test_features`.
